In [ ]:
#!g1.1
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 8.9 MB/s 


In [ ]:
! git clone https://github.com/tojiboyevf/image_captioning.git
! mv  -v /content/image_captioning/* /content/  

Cloning into 'image_captioning'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 432 (delta 132), reused 171 (delta 72), pack-reused 185
Receiving objects: 100% (432/432), 7.95 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (226/226), done.
renamed '/content/image_captioning/datasets' -> '/content/datasets'
renamed '/content/image_captioning/environment.yml' -> '/content/environment.yml'
renamed '/content/image_captioning/examples' -> '/content/examples'
renamed '/content/image_captioning/examples_otherfiles' -> '/content/examples_otherfiles'
renamed '/content/image_captioning/glove.py' -> '/content/glove.py'
renamed '/content/image_captioning/__init__.py' -> '/content/__init__.py'
renamed '/content/image_captioning/LICENSE' -> '/content/LICENSE'
renamed '/content/image_captioning/load_coco.sh' -> '/content/load_coco.sh'
renamed '/content/image_captioning/load_flickr8k.sh' -> '/c

In [ ]:
#!g1.1
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#!g1.1
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
from utils_torch import check_create_dir
from metrics import *
from utils_torch import words_from_tensors_fn
import numpy as np

In [ ]:
#!g1.1
import os.path
import pickle

from datasets.flickr8k import Flickr8kDataset


DATASET_BASE_PATH = 'data/flickr8k/'
VOCAB = 'vocab_set.pkl'

train_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='train',
    device=device, return_type='tensor', load_img_to_memory=False)


vocab_set = train_set.get_vocab()
with open(VOCAB, 'wb') as f:
    pickle.dump(vocab_set, f)
    

vocab, word2idx, idx2word, max_len = vocab_set
vocab_size = len(vocab)

# val_set_loss = Flickr8kDataset(
#     dataset_base_path=DATASET_BASE_PATH, dist='valloss',  vocab_set=vocab_set,
#     device=device, return_type='tensor', load_img_to_memory=False)

val_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='val', vocab_set=vocab_set,
    device=device, return_type='corpus', load_img_to_memory=False)

test_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='test', vocab_set=vocab_set,
    device=device, return_type='corpus', load_img_to_memory=False)

train_eval_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='train', vocab_set=vocab_set,
    device=device, return_type='corpus', load_img_to_memory=False)


print(
    f"The number of samples in:\ntrain: {len(train_set)};"
    + f" validation: {len(val_set)}; test: {len(test_set)}\n"
    + f"Vocabulary size: {vocab_size}; Max length of a sentence: {max_len};"
)

The number of samples in:
train: 30000; validation: 1000; test: 1000
Vocabulary size: 7708; Max length of a sentence: 40;


In [ ]:
#!g1.1
from torchvision import transforms
from torch.utils.data import DataLoader


BATCH_SIZE = 50


train_transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.RandomCrop(224),  # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])

eval_transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.CenterCrop(224),  # get 224x224 crop from random location
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])

train_set.transformations = train_transformations
val_set.transformations = eval_transformations
test_set.transformations = eval_transformations
train_eval_set.transformations = eval_transformations
# val_set_loss.transformations = eval_transformations

eval_collate_fn = lambda batch: (torch.stack([x[0] for x in batch]), [x[1] for x in batch], [x[2] for x in batch])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, sampler=None, pin_memory=False)
# val_loader_loss = DataLoader(val_set_loss, batch_size=BATCH_SIZE, shuffle=True, sampler=None, pin_memory=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                        collate_fn=eval_collate_fn)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                         collate_fn=eval_collate_fn)
train_eval_loader = DataLoader(train_eval_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                               collate_fn=eval_collate_fn)


In [ ]:
#!g1.1
start_token = word2idx['<start>']
end_token = word2idx['<end>']
pad_token = word2idx['<pad>']
max_seq_len = max_len

In [ ]:
#!g1.1
import random
import math
import torch
from torch import nn as nn
from torch.nn import functional as F
import torchvision.models as models
import torchvision.models as models
import numpy as np

np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

def embedding_layer(trainable=True, embedding_matrix=None, **kwargs):
    emb_layer = nn.Embedding(**kwargs)
    if embedding_matrix is not None:
        emb_layer.weight = nn.Parameter(torch.from_numpy(embedding_matrix).float())
    trainable = (embedding_matrix is None) or trainable
    if not trainable:
        emb_layer.weight.requires_grad = False
    return emb_layer


class Encoder(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        vgg16 = models.vgg16(pretrained=True)
        modules = list(vgg16.children())[:-1]
        self.vgg16 = nn.Sequential(*modules)
        self.embed = nn.Sequential(
            nn.Linear(512, embed_size),
            nn.Dropout(p=0.5),
        )
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
    
    def forward(self, images):
        with torch.no_grad():
            features = self.vgg16(images)
            features = F.avg_pool2d(features, kernel_size=7).view(features.size(0), -1)
        features = self.embed(features)
        features = self.bn(features)
        return features

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=40):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        

    def forward(self, x):
        if self.pe.size(0) < x.size(0):
            self.pe = self.pe.repeat(x.size(0), 1, 1).to(device)
        self.pe = self.pe[:x.size(0), : , : ]
        
        x = x + self.pe
        return self.dropout(x)

class Decoder(nn.Module):
    def __init__(
        self,
        num_heads,
        decoder_layers,
        embed_size, 
        vocab_size, 
        embedding_matrix=None, 
        train_embd=True, 
        max_len=40, 
        dropout=0.1
    ):
        super(Decoder, self).__init__()
        
        self.embedding = embedding_layer(num_embeddings=vocab_size, embedding_dim=embed_size,
                                     embedding_matrix=embedding_matrix, trainable=train_embd)
        self.pos_encoder = PositionalEncoding(embed_size, dropout)

        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embed_size, nhead=num_heads)
        self.transformer = nn.TransformerDecoder(self.decoder_layer, num_layers=decoder_layers)
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.embedding_size = embed_size
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc_out.bias.data.zero_()
        self.fc_out.weight.data.uniform_(-initrange, initrange)
    
    def generate_Mask(self, size, decoder_inp):
        decoder_input_mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        decoder_input_mask = decoder_input_mask.float().masked_fill(decoder_input_mask == 0, float('-inf')).masked_fill(decoder_input_mask == 1, float(0.0))

        decoder_input_pad_mask = decoder_inp.float().masked_fill(decoder_inp == 0, float(0.0)).masked_fill(decoder_inp > 0, float(1.0))
        decoder_input_pad_mask_bool = decoder_inp == 0

        return decoder_input_mask, decoder_input_pad_mask, decoder_input_pad_mask_bool
    

    def forward(self, features, captions):
        features = features.permute(1,0,2)
        decoder_inp_embed = self.embedding(captions)* math.sqrt(self.embedding_size)
        
        decoder_inp_embed = self.pos_encoder(decoder_inp_embed)
        decoder_inp_embed = decoder_inp_embed.permute(1,0,2)
        

        decoder_input_mask, decoder_input_pad_mask, decoder_input_pad_mask_bool = self.generate_Mask(captions.size(1), captions)
        decoder_input_mask = decoder_input_mask.to(device)
        decoder_input_pad_mask = decoder_input_pad_mask.to(device)
        decoder_input_pad_mask_bool = decoder_input_pad_mask_bool.to(device)
        

        decoder_output = self.transformer(tgt = decoder_inp_embed, memory = features, tgt_mask = decoder_input_mask, tgt_key_padding_mask = decoder_input_pad_mask_bool)
        
        final_output = self.fc_out(decoder_output)
        
        return final_output,  decoder_input_pad_mask
    
    def sample(self, features, max_len=40, topk=3, start_token=59, end_token=57, pad_token=58):
        input_seq = torch.ones(features.size(0), max_len).type(torch.LongTensor) * pad_token
        input_seq[:, 0] = start_token
        input_seq = input_seq.to(device)
        for i in range(max_len-1):
            output, _ = self.forward(features, input_seq)
            output = output[i, :, :]
            predicted = output.argmax(1)
            input_seq[:, i+1] = predicted
        return input_seq


class Captioner(nn.Module):
    def __init__(self, num_heads, decoder_layers, embed_size, vocab_size, embedding_matrix=None, train_embd=True):
        super().__init__()
        self.encoder = Encoder(embed_size)
        self.decoder = Decoder(num_heads, decoder_layers, embed_size, vocab_size,
                               embedding_matrix=embedding_matrix, train_embd=train_embd)

    def forward(self, images, captions):
        features = self.encoder(images)
        features = features.unsqueeze(1)
        output, padding_mask = self.decoder(features, captions)
        return output, padding_mask
    
    def sample(self, images, max_len=max_seq_len, topk=3, start_token=start_token, end_token=end_token, pad_token=pad_token):
        features = self.encoder(images)
        features = features.unsqueeze(1)
        captions = self.decoder.sample(features=features, max_len=max_len, topk=topk,
                                       start_token=start_token, end_token=end_token,
                                       pad_token=pad_token)
        return captions

In [ ]:
#!g1.1
from glove import embedding_matrix_creator
EMBEDDING_DIM = 50
EMBEDDING = f"GLV{EMBEDDING_DIM}"

embedding_matrix = embedding_matrix_creator(embedding_dim=EMBEDDING_DIM, word2idx=word2idx)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

  0%|          | 0/7708 [00:00<?, ?it/s]

Embedding matrix shape: (7708, 50)


In [ ]:
#!g1.1
final_model = Captioner(10, 6, EMBEDDING_DIM, vocab_size, embedding_matrix, False).to(device)

In [ ]:
#!g1.1 
import torch
from torch.nn.utils.rnn import pack_padded_sequence

from tqdm.auto import tqdm

def train_model(train_loader, model, loss_fn, optimizer, desc='', log_interval=25):
    running_loss = 0.0
    total_train_words = 0
    model.train()
    t = tqdm(iter(train_loader), desc=f'{desc}')
    for batch_idx, batch in enumerate(t):
        images, captions, lengths = batch
        sort_ind = torch.argsort(lengths, descending=True)
        images = images[sort_ind]
        captions = captions[sort_ind]
        lengths = lengths[sort_ind]

        optimizer.zero_grad()
        output, padding_mask = model(images, captions)
        output = output.permute(1, 2, 0)
        target = torch.cat([captions[:, 1:], pad_token * torch.ones(captions.size(0), 1).type(torch.LongTensor).to(device)], dim=1)

        loss = loss_fn(output, target)
        loss_masked = torch.mul(loss, padding_mask)
        final_batch_loss = torch.sum(loss_masked)/torch.sum(padding_mask)
        final_batch_loss.backward()
        optimizer.step()
        running_loss += torch.sum(loss_masked).detach().item()
        total_train_words += torch.sum(padding_mask).detach().item()
        t.set_postfix({'loss': running_loss / total_train_words}, refresh=True)
        if (batch_idx + 1) % log_interval == 0:
            print(f'{desc} {batch_idx + 1}/{len(train_loader)} '
                  f'train_loss: {running_loss / total_train_words:.4f} ')

    return running_loss / total_train_words
  
def validate_model(val_loader, model, loss_fn, desc='', log_interval=25):
    running_loss = 0.0
    total_val_words = 0
    model.train()
    t = tqdm(iter(val_loader), desc=f'{desc}')
    for batch_idx, batch in enumerate(t):
        images, captions, lengths = batch
        sort_ind = torch.argsort(lengths, descending=True)
        images = images[sort_ind]
        captions = captions[sort_ind]
        lengths = lengths[sort_ind]

        output, padding_mask = model(images, captions)
        output = output.permute(1, 2, 0)
        target = torch.cat([captions[:, 1:], pad_token * torch.ones(captions.size(0), 1).type(torch.LongTensor).to(device)], dim=1)

        loss = loss_fn(output, target)
        loss_masked = torch.mul(loss, padding_mask)
        
        running_loss += torch.sum(loss_masked).detach().item()
        total_val_words += torch.sum(padding_mask).detach().item()
        t.set_postfix({'loss': running_loss / total_val_words,
                       }, refresh=True)
        if (batch_idx + 1) == len(val_loader):  ### Можешь это удалить думаю в вал не нужен
            print(f'{desc} {batch_idx + 1}/{len(val_loader)} '
                  f'val_loss: {running_loss / total_val_words:.4f} ')

    return running_loss / total_val_words


def evaluate_model(data_loader, model, vocab_size, bleu_score_fn, tensor_to_word_fn, desc=''):
    running_bleu = [0.0] * 5
    model.eval()
    t = tqdm(iter(data_loader), desc=f'{desc}')
    for batch_idx, batch in enumerate(t):
        images, captions, lengths = batch
        outputs = tensor_to_word_fn(model.sample(images).cpu().numpy())

        for i in (1, 2, 3, 4):
            running_bleu[i] += bleu_score_fn(captions, outputs, n=i)
        t.set_postfix({
            'bleu1': running_bleu[1] / (batch_idx + 1),
            'bleu4': running_bleu[4] / (batch_idx + 1),
        }, refresh=True)
    for i in (1, 2, 3, 4):
        running_bleu[i] /= len(data_loader)
    return running_bleu


In [ ]:
#!g1.1



In [ ]:
#!g1.1 #embed=200, batch=50, batch,act, adam with weight_decay
MODEL = 'VGG16_transformer'
train_from_scratch = True

if train_from_scratch:
    check_create_dir(f'saved_models/{MODEL}')
    MODEL_NAME = f'saved_models/{MODEL}/{MODEL}_b{BATCH_SIZE}_emd{EMBEDDING}'
    NUM_EPOCHS = 5
    print(MODEL_NAME)

    
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=train_set.pad_value)
    acc_fn = accuracy_fn(ignore_value=train_set.pad_value)
    optimizer = torch.optim.Adam(final_model.parameters(), lr=0.0005, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.8, patience=2, verbose = True)
    min_val_loss = np.float('Inf')

    sentence_bleu_score_fn = bleu_score_fn(4, 'sentence')
    corpus_bleu_score_fn = bleu_score_fn(4, 'corpus')
    tensor_to_word_fn = words_from_tensors_fn(idx2word=idx2word)
    train_loss_min = np.float('Inf')
    val_bleu4_max = 0.0

    for epoch in range(NUM_EPOCHS):
        train_loss = train_model(
            desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}',
            model=final_model,                                                                      
            optimizer=optimizer,
            loss_fn=loss_fn,                                        
            train_loader=train_loader
        )

        with torch.no_grad():
            train_bleu = evaluate_model(
                desc=f'\tTrain Bleu Score: ',
                model=final_model,
                bleu_score_fn=corpus_bleu_score_fn,
                tensor_to_word_fn=tensor_to_word_fn,
                data_loader=train_eval_loader,
                vocab_size=vocab_size
            )
            val_bleu = evaluate_model(
                desc=f'\tValidation Bleu Score: ',
                model=final_model,
                bleu_score_fn=corpus_bleu_score_fn,
                tensor_to_word_fn=tensor_to_word_fn,
                data_loader=val_loader, 
                vocab_size=vocab_size
            )

            print(f'Epoch {epoch + 1}/{NUM_EPOCHS}',
                ''.join([f'train_bleu{i}: {train_bleu[i]:.4f} ' for i in (1, 4)]),
                ''.join([f'val_bleu{i}: {val_bleu[i]:.4f} ' for i in (1, 4)]),
                )
            
            state = {
                'epoch': epoch + 1,
                'state_dict': final_model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'train_loss_latest': train_loss,
                'val_bleu4_latest': val_bleu[4],
                'train_loss_min': min(train_loss, train_loss_min),
                'val_bleu4_max': max(val_bleu[4], val_bleu4_max),
                'train_bleus': train_bleu,
                'val_bleus': val_bleu
            }

            torch.save(state, f'{MODEL_NAME}_latest.pt')

            if train_loss < train_loss_min:
                train_loss_min = train_loss
                torch.save(state, f'{MODEL_NAME}''_best_train_loss.pt')
                
            if val_bleu[4] > val_bleu4_max:
                val_bleu4_max = val_bleu[4]
                torch.save(state, f'{MODEL_NAME}''_best_val_bleu.pt')
        scheduler.step(train_loss)

    torch.save(state, f'{MODEL_NAME}_ep{NUM_EPOCHS:02d}_weights.pt')

saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 1/5:   0%|          | 0/600 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
#!g1.1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -av '/content/saved_models/VGG16_transformer' "/content/drive/MyDrive/DL/Project"

'/content/saved_models/VGG16_transformer' -> '/content/drive/MyDrive/DL/Project/VGG16_transformer'
'/content/saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV200_latest.pt' -> '/content/drive/MyDrive/DL/Project/VGG16_transformer/VGG16_transformer_b50_emdGLV200_latest.pt'
'/content/saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV200_best_train_loss.pt' -> '/content/drive/MyDrive/DL/Project/VGG16_transformer/VGG16_transformer_b50_emdGLV200_best_train_loss.pt'
'/content/saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV200_best_val_bleu.pt' -> '/content/drive/MyDrive/DL/Project/VGG16_transformer/VGG16_transformer_b50_emdGLV200_best_val_bleu.pt'
'/content/saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV50_latest.pt' -> '/content/drive/MyDrive/DL/Project/VGG16_transformer/VGG16_transformer_b50_emdGLV50_latest.pt'
'/content/saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV50_best_train_loss.pt' -> '/content/drive/MyDrive/DL/Project/VGG16_tra

In [ ]:
!gdown --id 1-38rLF1EvS6IDolOSdqb4295Vb8jgQS6

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-38rLF1EvS6IDolOSdqb4295Vb8jgQS6
To: /content/VGG16_transformer_b50_emdGLV200_best_val_bleu.pt
100% 174M/174M [00:01<00:00, 94.6MB/s]


In [ ]:
if not train_from_scratch:
    MODEL_DIR = './'
    MODEL_NAME = f'{model_name}_b{batch_size}_emd{embedding_desc}_best_val.pt'
    
    if model_name == "vgg16_lstm":
        !gdown 1_e4xb1BZuh_aS7171BRQm7FipbgqJJoz
    
    elif model_name == "densenet161_lstm":
        !gdown 1CLODHygvcd9Lfczy5bsUT656OkXOTvUG
    
    checkpoint = torch.load(os.path.join(MODEL_DIR, MODEL_NAME), map_location=torch.device('cpu'))
    !rm $MODEL_NAME 

    # define the model:
    model = Captioner(
        embedding_dim,
        hidden_size,
        vocab_size,
        num_layers=num_layers,
        embedding_matrix=embedding_matrix,
        train_embd=train_embd
    ).to(device)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()

In [ ]:
PATH = '/content/saved_models/VGG16_transformer/VGG16_transformer_b50_emdGLV50_best_val_bleu.pt'
final_model.load_state_dict(torch.load(PATH, map_location=torch.device(device))['state_dict'])
final_model.eval()

Captioner(
  (encoder): Encoder(
    (vgg16): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace=True)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:
torch.load(PATH, map_location=torch.device(device))['val_bleus'], torch.load(PATH, map_location=torch.device(device))['train_bleus']

([0.0,
  0.5279028008819513,
  0.3307541635152168,
  0.20134296556512604,
  0.1231079127594675],
 [0.0,
  0.5313176308552883,
  0.3363814838274679,
  0.21012488737622093,
  0.1321288381144432])

In [ ]:
inter_params = {
    'model': final_model,
    'bleu_score_fn': corpus_bleu_score_fn,
    'tensor_to_word_fn': tensor_to_word_fn,
    'vocab_size': vocab_size, 
}

In [ ]:
with torch.no_grad():
    train_bleu = evaluate_model(
        desc=f'Train: ',
        data_loader=train_eval_loader,
        **inter_params,
    )
    
    val_bleu = evaluate_model(
        desc=f'Val: ',
        data_loader=val_loader,
        **inter_params,
    )
    
    test_bleu = evaluate_model(
        desc=f'Test: ',
        data_loader=test_loader,
        **inter_params,
    )
    for setname, result in zip(('train', 'val', 'test'), (train_bleu, val_bleu, test_bleu)):
        print(setname, end=' ')
        for ngram in (1, 2, 3, 4):
            print(f'Bleu-{ngram}: {result[ngram]}\n', end=' ')
        print()

Train:   0%|          | 0/120 [00:00<?, ?it/s]

Val:   0%|          | 0/20 [00:00<?, ?it/s]

Test:   0%|          | 0/20 [00:00<?, ?it/s]

train Bleu-1: 0.5313176308552883
 Bleu-2: 0.3363814838274679
 Bleu-3: 0.21012488737622093
 Bleu-4: 0.1321288381144432
 
val Bleu-1: 0.5279028008819513
 Bleu-2: 0.3307541635152168
 Bleu-3: 0.20134296556512604
 Bleu-4: 0.1231079127594675
 
test Bleu-1: 0.5276105108829945
 Bleu-2: 0.3304151787552659
 Bleu-3: 0.20273477808085913
 Bleu-4: 0.12388007698440762
 


In [ ]:
import nltk
import os
import torch.utils.data as data
from vocabulary import Vocabulary
from PIL import Image
from pycocotools.coco import COCO
import numpy as np
from tqdm import tqdm


def CoCoDataloader(transform,
               batch_size=1,
               vocab_threshold=None,
               vocab_file='./vocab_set.pkl',
               start_word="<start>",
               end_word="<end>",
               unk_word="<unk>",
               vocab_from_file=True,
               size=0.1,
               num_workers=2,
               img_folder='data/coco/val2014',
               annotations_file='data/coco/annotations/captions_val2014.json',
               shuffle=True,
               random_seed=42):
    
    

    dataset = CoCoDataset(transform=transform,
                          batch_size=batch_size,
                          vocab_threshold=vocab_threshold,
                          vocab_file=vocab_file,
                          start_word=start_word,
                          end_word=end_word,
                          unk_word=unk_word,
                          annotations_file=annotations_file,
                          vocab_from_file=vocab_from_file,
                          img_folder=img_folder)

    indices = dataset.get_train_indices()

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(size * dataset_size))

    if shuffle :
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    indices = indices[:split]

    initial_sampler = data.sampler.SubsetRandomSampler(indices)

    data_loader = data.DataLoader(dataset=dataset, 
                                    num_workers=num_workers,
                                    batch_sampler=data.sampler.BatchSampler(sampler=initial_sampler,
                                                                            batch_size=dataset.batch_size,
                                                                            drop_last=False))

    return data_loader

class CoCoDataset(data.Dataset):
    
    def __init__(self, transform, batch_size, vocab_threshold, vocab_file, start_word, 
        end_word, unk_word, annotations_file, vocab_from_file, img_folder):
        self.transform = transform
        self.batch_size = batch_size
        self.vocab = Vocabulary(vocab_threshold, vocab_file, start_word,
            end_word, unk_word, annotations_file, vocab_from_file)
        self.img_folder = img_folder

        self.coco = COCO(annotations_file)
        self.ids = list(self.coco.anns.keys())
        print('Obtaining caption lengths...')
        all_tokens = [nltk.tokenize.word_tokenize(str(self.coco.anns[self.ids[index]]['caption']).lower()) for index in tqdm(np.arange(len(self.ids)))]
        self.caption_lengths = [len(token) for token in all_tokens]

        
    def __getitem__(self, index):

        ann_id = self.ids[index]
        caption = self.coco.anns[ann_id]['caption']
        img_id = self.coco.anns[ann_id]['image_id']
        path = self.coco.loadImgs(img_id)[0]['file_name']

        image = Image.open(os.path.join(self.img_folder, path)).convert('RGB')
        # orig_image = np.array(image)
        image = self.transform(image)

        return image, caption

    def get_train_indices(self):
        sel_length = np.random.choice(self.caption_lengths)
        all_indices = np.where([self.caption_lengths[i] == sel_length for i in np.arange(len(self.caption_lengths))])[0]
        indices = list(np.random.choice(all_indices, size=self.batch_size))
        return indices

    def __len__(self):
        return len(self.ids)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
vocab_from_file = True # if True, load existing vocab file
split_size = 1 # 1 means the whole data

coco_val_loader = CoCoDataloader(
    transform=eval_transformations,
    batch_size=200,
    vocab_from_file=vocab_from_file,
    vocab_file='./vocab_set.pkl',
    size=split_size,
    img_folder='data/coco/val2014',
    annotations_file='data/coco/annotations/captions_val2014.json',
    shuffle=True,
    random_seed=42,
)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 202654/202654 [00:23<00:00, 8451.30it/s]


In [ ]:
def evaluate_model_coco(data_loader, model, data, bleu_score_fn, tensor_to_word_fn, desc=''):
    running_bleu = [0.0] * 5
    model.eval()
    t = tqdm(iter(data_loader), desc=f'{desc}')
    for batch_idx, batch in enumerate(t):
        if data=='coco':
            images, caption_list = batch
            captions = []
            for caption in caption_list:
              toks = nltk.word_tokenize(caption.lower())
              captions.append([toks])       
            images = images.to(device)
        else:
            images, captions, _ = batch
        
        outputs = tensor_to_word_fn(model.sample(images).cpu().numpy())

        for i in (1, 2, 3, 4):
            running_bleu[i] += bleu_score_fn(captions, outputs, n=i)
        t.set_postfix({
            'bleu1': running_bleu[1] / (batch_idx + 1),
            'bleu4': running_bleu[4] / (batch_idx + 1),
        }, refresh=True)
    for i in (1, 2, 3, 4):
        running_bleu[i] /= len(data_loader)
    return running_bleu

In [ ]:
with torch.no_grad():
    val_bleu = evaluate_model_coco(
        desc=f'\tValidation Bleu Score: ',
        model=final_model,
        data='coco',
        bleu_score_fn=corpus_bleu_score_fn,
        tensor_to_word_fn=tensor_to_word_fn,
        data_loader=coco_val_loader,
    )
    print()
    print('val', end=' ')
    for ngram in (1, 2, 3, 4):
        print(f'Bleu-{ngram}: {val_bleu[ngram]}', end=' ')
    print()

	Validation Bleu Score: 100%|██████████| 1014/1014 [35:43<00:00,  2.11s/it, bleu1=0.255, bleu4=0.0204]


val Bleu-1: 0.25549095888028245 Bleu-2: 0.10037266442048844 Bleu-3: 0.04107841183757298 Bleu-4: 0.020401689097716107 
